In [ ]:
# # 1. Region proposals
# - selective search
# # 2. CNN
# - IoU를 통한 샘플 분류
# - pre-train된 Alexnet 수정
# # 3. Linear SVM
# - Hard Negaive Mining
# - BB Regressor
# - NMS

In [23]:
import numpy as np

In [53]:
img = cv2.imread('./car.jpg') #channel_last
tar_bb = [156, 97, 351, 270]

In [28]:
## 1. Region proposals
# selective search
import cv2

sel_search = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
sel_search.setBaseImage(img)
sel_search.switchToSelectiveSearchFast() # 2000개 내외
pps_bb = sel_search.process() # 좌표생성
pps_bb[:, 2] += pps_bb[:, 0] 
pps_bb[:, 3] += pps_bb[:, 1]

In [45]:
# # 2. CNN
# - IoU를 통한 샘플 분류
# IoU가 0.5보다 크면 pos_sample/neg_sample로 분류

def IoU(tar_bb, pps_bb):
    # bb = (xmin, ymin, xmax, ymax)
    tar_area = (tar_bb[2]-tar_bb[0]) * (tar_bb[3]-tar_bb[1])
    pps_area = (pps_bb[2]-pps_bb[0]) * (pps_bb[3]-pps_bb[1])

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(tar_bb[0], pps_bb[0])
    y1 = max(tar_bb[1], pps_bb[1])
    x2 = min(tar_bb[2], pps_bb[2])
    y2 = min(tar_bb[3], pps_bb[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (tar_area+pps_area-inter)
    return iou

In [48]:
# pps_bb 별 IoU를 통해 샘플 분류
pos_sample = []
neg_sample = []
for i in pps_bb:
    if IoU(tar_bb, i) < 0.5:
        neg_sample.append(i)
    else:
        pos_sample.append(i)
    

In [72]:
# fine-tuning을 위해서 pre-train된 AlexNet을 불러와 수정
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
num_classes = 5
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(4096, num_classes+1)

In [75]:
model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=6, bias=True)
)

In [ ]:
# 32(pos) + 96(neg) = 128개를 하나의 batch로 fine-tuning 과정 진행

In [ ]:
# # 3. Linear SVM
# - Hard Negaive Mining
# - BB Regressor
# - NMS